# Introduction

*Vous pouvez retrouver le lien vers le projet Github [ici](https://github.com/Mathieu2301/Projet-BioInfo).*

Cette partie importe toutes les libraies nécessiares pour les parties A, B, C, D et E.

Le dictionnaire `SEQUENCES` représente la liste des séquences que l'on souhaite analyser et leurs requêtes associées.

In [1]:
from Bio import Entrez
from Bio import SeqIO
# On importe les module permettant d'aligner les séquences
from Bio.Align.Applications import MafftCommandline
from os import path
from platform import system

# On définit les séquences qu'on veut récupérer
# On les nomme en indice pour savoir quelle est la première, la seconde, etc...
# La valeur de chacun est la requête correspondente.
SEQUENCES = {
  'HOMME': 'SARS-Cov2 [ORGN] AND srcdb_refseq [PROP]',
  'CHAUVE-SOURIS': 'SARSr-CoV RaTG13',
  'PANGOLIN': 'Pangolin coronavirus isolate MP789 MT121216',
}

## Partie A

On utilise `Entrez` de la librarie `BioPython` pour exécuter les requêtes et récupérer les séquences.

In [2]:
Entrez.email = 'mathieu@colmon.fr'

# On fait une fonction pour récupérer le premier
# résultat d'une requête sous forme d'un objet Seq
def getSeq(request):
  # On exécute la requête demandée
  searchReq = Entrez.esearch(db='nucleotide', term=request)
  # On lit le résultat
  searchRes = Entrez.read(searchReq)
  # On ferme la requête
  searchReq.close()

  # On demande le premier résultat
  firstReq = Entrez.efetch(db='nucleotide', id=searchRes['IdList'], rettype='gb')
  # On lit le résultat
  firstRes = SeqIO.read(firstReq, 'gb')
  # On ferme la requête
  firstReq.close()

  return firstRes

# On crée une liste qui stockera les séquences
SEQs = []

# Pour chaque séquence demandée
for sName in SEQUENCES:
  # On traite la requête et on stocke le résustat dans 'SEQs'
  SEQs.append(getSeq(SEQUENCES[sName]))

# On écrit le résultat dans le fichier 'seq_covid.gb'
SeqIO.write(SEQs, './sequences/seq_covid.gb', 'genbank')

# Log #
print('Séquences des coronavirus écrites dans le fichier \'./sequences/seq_covid.gb\' !')

Séquences des coronavirus écrites dans le fichier './sequences/seq_covid.gb' !


## Partie B

In [3]:
SEQs = SeqIO.parse('./sequences/seq_covid.gb', 'genbank')

SPIKEs = []

# Pour chaque séquence de coronavirus
for seq in SEQs:
  for f in seq.features:
    # Si le type est 'gene', on vérifie si son nom est 'S'
    if f.type == 'gene' and f.qualifiers.get('gene')[0] == 'S':
      # On ajoute à la liste de séquences 'SPIKEs'
      SPIKEs.append(f.translate(seq))

# On écrit le contenu de 'SPIKEs' dans le fichier 'spike.fasta'
SeqIO.write(SPIKEs, './sequences/spike.fasta', 'fasta')

# Log #
print('Séquences des gènes écrites dans le fichier \'./sequences/spike.fasta\' !')

Séquences des gènes écrites dans le fichier './sequences/spike.fasta' !


## Partie C

In [4]:
# Par défaut (sur Linux) le moyen d'accéder à MAFFT est la commande 'mafft'
mafftPath = 'mafft'

# Sur Windows, MAFFT n'est accessible depuis la commande 'mafft'.
if system()== 'Windows':
  # On cherche donc le chemin d'accès absolu du fichier './mafft-win/mafft.bat'
  mafftPath = path.abspath('./mafft-win/mafft.bat')

# On Aligne les séquences du fichier 'spike.fasta'
stdout, stderr = MafftCommandline(cmd=mafftPath, input='./sequences/spike.fasta')()

# On écrit le résultat dans 'aln-spike.fasta'
with open('./sequences/aln-spike.fasta', 'w') as w:
  w.write(stdout)

# Log #
print('Séquences alignées dans le fichier \'./sequences/aln-spike.fasta\' !')

Séquences alignées dans le fichier './sequences/aln-spike.fasta' !


## Partie D

In [5]:
# On récupère toutes les séquences alignées
SEQs = SeqIO.parse('./sequences/aln-spike.fasta', 'fasta')

# On crée une liste qui contiendra les séquences sous forme de chaines de caractères
rawSeqs = []

# Pour chaque séquence alignée
for seq in SEQs:
  # On stocke la séquence sous forme de chaine de caractère dans 'rawSeqs'
  rawSeqs.append(seq.seq)

# On crée une liste qui contiendra les noms des colonnes du tableau résultat
indexes = list(SEQUENCES.keys())
# La première colonne est celle de la position
indexes.insert(0, 'POSITION')

# On ouvre le fichier qui contiendra les résultats des comparaisons
file = open('./sequences/resultatComparaison_geneS.txt', 'w')

# On stocke le nombre de lignes écrites dans une variable de manière à
# pouvoir afficher toutes les n (50) lignes les noms des indices
nbLines = 0

# On crée une fonction qui permet à la fois d'afficher une ligne
# dans la console, et de stocker la ligne dans le fichier résultat
def printTableLine(data):
  # On importe la variable globale 'nbLines'
  global nbLines
  # On incrémente la variable 'nbLines'
  nbLines += 1
  # Si l'indice de la ligne (nbLines - 1) est un multiple de 50
  
  if (nbLines - 1) % 50 == 0:
    # On affiche et stocke les indices
    printTableLine(indexes)

  # On formatte la ligne du tableau
  formattedRow = '{: >8} {: >15} {: >15} {: >15}'.format(*data)
  # On l'écrit dans le fichier résultat
  file.write(formattedRow + '\n')
  # On l'écrit dans la console
  print(formattedRow)

# Les séquences sont alignées donc elles ont le même nombre de caractères.
# Pour chaque caractère de la (première) séquence
for pos in range(len(rawSeqs[0])):
  # Pour chaque séquence...
  for rawSeq in rawSeqs:
    # S'il y a une différence du n ième caractère entre
    # la première séquence et la séquence 'rawSeq'
    if rawSeq[pos] != rawSeqs[0][pos]:
      # On crée une liste qui correspond au changement du n ième caractère
      change = [pos]
      # Pour chaque indice de la liste des séquences alignées
      for iSeq in range(len(rawSeqs)):
        # On ajoute les changements dans la liste
        change.append(rawSeqs[iSeq][pos])

      # On ajoute la ligne au tableau des changements
      printTableLine(change)
      # On passe au caractère suivant
      break

# On ferme le fichier
file.close()

POSITION           HOMME   CHAUVE-SOURIS        PANGOLIN
       0               -               -               M
       1               M               M               L
       3               V               V               F
       5               L               L               F
       6               V               V               L
       7               L               L               H
       8               L               L               F
       9               P               P               A
      12               S               S               N
      20               T               T               G
      22               T               T               A
      23               Q               Q               A
      24               L               L               I
      25               P               P               Q
      27               A               A               S
      28               Y               Y               F
      32               F       

## Partie E

In [6]:
# On récupère toutes les séquences alignées
SEQs = SeqIO.parse('./sequences/aln-spike.fasta', 'fasta')

# On crée une liste qui contiendra les séquences sous forme de chaines de caractères
rawSeqs = []

# Pour chaque séquence alignée
for seq in SEQs:
  # On stocke la séquence sous forme de chaine de caractère dans 'rawSeqs'
  rawSeqs.append(seq.seq)

# On crée un tableau de tableaux contant le nom de chaque espèces en premier indice, et la valeur correspondant aux différence en second indice
# La première espèce étudié est considiré comme celle de référence pour calculer le pourcentage
conservationCounter = [ [name, 0] for name in (SEQUENCES.keys())]
# On calcule le nombre de lettres totale dans l'humain
conservationCounter[0][1] = len(rawSeqs[0])

# Les séquences sont alignées donc elles ont le même nombre de caractères.
# Pour chaque caractère de la (première) séquence
for pos in range(conservationCounter[0][1]):
  # Pour chaque séquence...
  for i in range(len(rawSeqs[1:])):
    rawSeq = rawSeqs[1:][i]
    # S'il y a une différence du n ième caractère entre
    # la première séquence (séquence comparée) et la séquence 'rawSeq' (séquence de référence) 
    if rawSeq[pos] == rawSeqs[0][pos]:
      # On incrémente de 1 le nombre de différences dans le tableau
      conservationCounter[1:][i][1] += 1

# Pour chaque espèces sans la première (espèce de référence) 
for id in conservationCounter[1:]:
  # On calcule le pourcentage par rapport à l'espèce de référance:
  pourcentage = round( (conservationCounter[0][1] - id[1]) * 100 / conservationCounter[0][1], 2)
  # On affiche le résultat avec le nom des deux espèces comparés
  print(f'Le pourcentage de différence entre {conservationCounter[0][0]} et {id[0]} est de {pourcentage}%.')

Le pourcentage de différence entre HOMME et CHAUVE-SOURIS est de 2.59%.
Le pourcentage de différence entre HOMME et PANGOLIN est de 9.96%.


## Partie F

Cette partie reprend l'ensemble des parties précédentes dans une fonction traitant les donnés automatiquement. Le code qui suit peut-être utilisé dans un autre projet de façon indépendante. La partie F ne contient volontairement que peu de commentaires, car le but est de pouvoir utiliser le code qui suit en production. Les différentes parties sont expliqués précédement dans ce même document.

In [7]:
from Bio import Entrez
from Bio import SeqIO
from Bio.Align.Applications import MafftCommandline
from os import path
from platform import system
from pathlib import Path

def analyzeProcess(dictionnary, geneLetters):
  print(f'| ----------------------------------------')
  print(f'| Début de l\'analyse')
  print(f'| Paramètres en entrée:')
  for a in dictionnary:
    print(f'| > {a} : {dictionnary[a]}')
  print(f'| --------------------')
  print(f'| Gènes analysés:')
  for a in geneLetters:
    print(f'| > Gène {a}')
  print(f'| ----------------------------------------\n')

  # Partie A / Création du fichier avec les séquences demandés
  Entrez.email = 'contact@antt0n.tech'

  def getSeq(request):
    searchReq = Entrez.esearch(db='nucleotide', term=request)
    searchRes = Entrez.read(searchReq)
    searchReq.close()

    firstReq = Entrez.efetch(db='nucleotide', id=searchRes['IdList'], rettype='gb')
    firstRes = SeqIO.read(firstReq, 'gb')
    firstReq.close()

    return firstRes

  SEQs = []
  for sName in SEQUENCES:
    SEQs.append(getSeq(SEQUENCES[sName]))
  Path('./sequences/Etude/').mkdir(parents=True, exist_ok=True)
  SeqIO.write(SEQs, './sequences/Etude/seq.gb', 'genbank')

  print(f'Séquences écrites dans le fichier \'/sequences/Etude/seq.gb\' !\n')

  for geneLetter in geneLetters:
    print(f'Début de l\'analyse pour le gène {geneLetter}.')

    # Partie B 
    Path(f'./sequences/Etude/Gene{geneLetter}').mkdir(parents=True, exist_ok=True)
    SEQs = SeqIO.parse(f'./sequences/Etude/seq.gb', 'genbank')

    GENE = []
    for seq in SEQs:
      for f in seq.features:
        if f.type == 'gene' and f.qualifiers.get('gene')[0] == geneLetter:
          GENE.append(f.translate(seq))

    SeqIO.write(GENE, f'./sequences/Etude/Gene{geneLetter}/gene{geneLetter}.fasta', 'fasta')

    print(f'Séquences des gènes écrites dans le fichier \'./sequences/Etude/Gene{geneLetter}/gene{geneLetter}.fasta\' !')

    # Partie C
    mafftPath = 'mafft'
    if system()== 'Windows':
      mafftPath = path.abspath('./mafft-win/mafft.bat')

    stdout, _ = MafftCommandline(cmd=mafftPath, input=f'./sequences/Etude/Gene{geneLetter}/gene{geneLetter}.fasta')()

    with open(f'./sequences/Etude/Gene{geneLetter}/aln-gene{geneLetter}.fasta', 'w') as w:
      w.write(stdout)

    print(f'Séquences alignées dans le fichier \'./sequences/Etude/Gene{geneLetter}/aln-gene{geneLetter}.fasta\' !')

    # Partie D
    print('Affichage du tableau de comparaisons :\n----------')

    SEQs = SeqIO.parse(f'./sequences/Etude/Gene{geneLetter}/aln-gene{geneLetter}.fasta', 'fasta')

    rawSeqs = []

    for seq in SEQs:
      rawSeqs.append(seq.seq)

    indexes = list(SEQUENCES.keys())
    indexes.insert(0, 'POSITION')

    file = open(f'./sequences/Etude/Gene{geneLetter}/resultatComparaison_gene{geneLetter}.txt', 'w')

    def printTableLine(data):
      formattedRow = '{: >8} {: >15} {: >15} {: >15}'.format(*data)
      file.write(formattedRow + '\n')
      print(formattedRow)

    printTableLine(indexes)

    for pos in range(len(rawSeqs[0])):
      for rawSeq in rawSeqs:
        if rawSeq[pos] != rawSeqs[0][pos]:
          change = [pos]
          for iSeq in range(len(rawSeqs)):
            change.append(rawSeqs[iSeq][pos])

          printTableLine(change)
          break

    print('----------')

    file.close()

    # Partie E
    conservationCounter = [ [name, 0] for name in (SEQUENCES.keys())]
    conservationCounter[0][1] = len(rawSeqs[0])

    for pos in range(conservationCounter[0][1]):
      for i in range(len(rawSeqs[1:])):
        rawSeq = rawSeqs[1:][i]
        if rawSeq[pos] == rawSeqs[0][pos]:
          conservationCounter[1:][i][1] += 1

    for id in conservationCounter[1:]:
      pourcentage = round((conservationCounter[0][1] - id[1]) * 100 / conservationCounter[0][1], 2)
      print(f'Le pourcentage de différence entre {conservationCounter[0][0]} et {id[0]} est de {pourcentage}% (gène {geneLetter}).')

    print(f'Fin de l\'analyse pour le gène {geneLetter}.')
    print('\n--------------------\n')

In [8]:
SEQUENCES = {
  'HOMME': 'SARS-Cov2 [ORGN] AND srcdb_refseq [PROP]',
  'CHAUVE-SOURIS': 'SARSr-CoV RaTG13',
  'PANGOLIN': 'Pangolin coronavirus isolate MP789 MT121216',
}

GENES = ['S', 'M', 'N', 'E']

analyzeProcess(SEQUENCES, GENES)

| ----------------------------------------
| Début de l'analyse
| Paramètres en entrée:
| > HOMME : SARS-Cov2 [ORGN] AND srcdb_refseq [PROP]
| > CHAUVE-SOURIS : SARSr-CoV RaTG13
| > PANGOLIN : Pangolin coronavirus isolate MP789 MT121216
| --------------------
| Gènes analysés:
| > Gène S
| > Gène M
| > Gène N
| > Gène E
| ----------------------------------------

Séquences écrites dans le fichier '/sequences/Etude/seq.gb' !

Début de l'analyse pour le gène S.
Séquences des gènes écrites dans le fichier './sequences/Etude/GeneS/geneS.fasta' !
Séquences alignées dans le fichier './sequences/Etude/GeneS/aln-geneS.fasta' !
Affichage du tableau de comparaisons :
----------
POSITION           HOMME   CHAUVE-SOURIS        PANGOLIN
       0               -               -               M
       1               M               M               L
       3               V               V               F
       5               L               L               F
       6               V              

## Partie G

Ces trois coronavirus semblent être particulièrement similaires.
Voici les résultats obtenus grace au code écrit précédemment:

- Gène M
    - Le pourcentage de différence entre HOMME et CHAUVE-SOURIS est de 0.9% (gène M).
    - Le pourcentage de différence entre HOMME et PANGOLIN est de 1.8% (gène M).
- Gène E
    - Le pourcentage de différence entre HOMME et CHAUVE-SOURIS est de 0.0% (gène E).
    - Le pourcentage de différence entre HOMME et PANGOLIN est de 0.0% (gène E).
- Gène N
    - Le pourcentage de différence entre HOMME et CHAUVE-SOURIS est de 0.95% (gène N).
    - Le pourcentage de différence entre HOMME et PANGOLIN est de 2.15% (gène N).
- Gène S
    - Le pourcentage de différence entre HOMME et CHAUVE-SOURIS est de 2.59% (gène S).
    - Le pourcentage de différence entre HOMME et PANGOLIN est de 9.96% (gène S)

Nous pouvons constater que dans l'ensemble, le pourcentage de différences entre ces trois coronavirus est relativement faible. 
Ce qui nous permet d'en conclure qu'ils sont relativement similaire.
Tout du moins, le coronavirus de la chauve-souris semble être plus proche du coronavirus humain que celui du pangolin.
La protéine qui semble le plus changer entre ces trois virus différents semble être la protéine spike (la différence de ces protéine entre l'humain et le pangolin monte à 9.96%, ce qui représente la valeur de différence la plus élevé toute protéine confondues).
A l'inverse, c'est le gène E qui ne change absolument pas entre ces trois virus.

## Rapport

### Informations sur le projet
 - **Composition du groupe:** Mathieu Colmon et Seguin Antoine (Groupe A / CMI).
 - **Technologies utilisées:** JupyterNotebook, Github.
 - **Language de programmation:** Python.
 - **Dépendances du projet:** Biopython, OS, plateform, pathlib (librairies Python).

### Nos démarches

Nous avons commencé par étudier le sujet et réfléchi au support sur lequel nous allions répondre aux différentes questions. Nous avons décidé d'utiliser *JupyterNotebook* pour l'ensemble du devoir, afin de faciliter le découpage du code et le travail en équipe.  
Un autre avantage de ce support est la possibilité d'ajouter des éléments textes, afin de structurer plus proprement notre projet, ainsi que sa grande compatibilité avec les outils de développement que nous avons utilisé sur nos ordinateurs, étant donné que nous avons fait le choix d'utiliser Github, pour gagner en productivité.  
Après avoir analysé ce qui nous était demandé, nous avons suivi point par point les différents énoncés. Nous avons essayé de concevoir le code dans un souci de "réutilisation". Le but étant de rendre le code le plus modulaire possible, en prévision de la **partie F**, mais aussi par un souci du détail, qui nous semblait relativement important.  
Nous avons également essayé de rendre le code le plus polyvalent possible, afin qu'il puisse être exécuté sous le plus d'environnements possibles, en limitant le plus possible les prérequis.  
C'est en ce souci que la **partie C** intègre une vérification de l'environnement utilisé lors de l'exécution du programme, car la commande `maftt` semblait ne pas fonctionner correctement sous certains OS.  
Dans la **partie D**, nous avons pris la liberté d'ajouter lors de l'affichage des résultats, un rappel du titre de chaque colonnes toutes les 50 lignes, afin de faciliter la lisibilité et l'interprétation des résultats.  
Pour finir, dans la **partie F**, nous avons fait le choix d'une fonction, afin de faciliter encore une fois la "réutilisation" du code. Nous avons également estimé que les commentaires n'étaient pas utiles, étant donné que nous avons simplement repris les différentes parties précédentes, que nous avons ensuite fait travailler entre elles. La lisibilité étant, de nouveau, un enjeu important d'après nous, nous avons essayé de rendre le résultat en console le plus propre possible, afin de faciliter l'exploitation des différents résultats.

### Comment exécuter le code ?

La démarche à suivre est indiqué dans le fichier `README.md`, disponible sur le repository Github du projet.